
Tensorflow is a framework for deep learning. It is a symbolic math library, and is also used for machine learning applications such as neural networks. One of the main thing it uses is the Computational Graph. The main thing it helps with is that we can split the graph into many parts and use our hardware across many servers to calculate it.

<div style="text-align: center;">
<img src="Images/ComGraph.png" alt="Alt text describing the image" width="500"/>
</div>


In [28]:
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.disable_eager_execution() # need to disable eager in TF2.
tf.config.run_functions_eagerly(True)

2.13.0


In [31]:
import tensorflow as tf
# Đây là các bước có thể xây dựng một computational graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
# Tuy nhiên, để có thể chạy nó thì cần phải có một session. Session ở đây được định nghĩa là một cách để chạy một computational graph.
with tf.compat.v1.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [29]:
x1 = tf.Variable(5)
print(x1.graph is tf.compat.v1.get_default_graph())

True
